In [0]:
# Uploaded Files 
display(dbutils.fs.ls("FileStore/tables/"))

path,name,size,modificationTime
dbfs:/FileStore/tables/OP_CVRD_RCPNT_PRFL_SPLMTL_README_P01302025.txt,OP_CVRD_RCPNT_PRFL_SPLMTL_README_P01302025.txt,3674,1742778323000
dbfs:/FileStore/tables/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv,OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv,1827342,1742779004000
dbfs:/FileStore/tables/OP_PGYR2023_README_P01302025.txt,OP_PGYR2023_README_P01302025.txt,5422,1742779013000
dbfs:/FileStore/tables/OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv,OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv,158460,1742778998000
dbfs:/FileStore/tables/PGYR2023_P01302025_01212025-1.zip,PGYR2023_P01302025_01212025-1.zip,789005271,1742779918000
dbfs:/FileStore/tables/PGYR2023_P01302025_01212025.zip,PGYR2023_P01302025_01212025.zip,789005271,1742776742000
dbfs:/FileStore/tables/PHPRFL_P01302025_01212025-1.zip,PHPRFL_P01302025_01212025-1.zip,82966770,1742779261000
dbfs:/FileStore/tables/PHPRFL_P01302025_01212025.zip,PHPRFL_P01302025_01212025.zip,82966770,1742776848000


Those are the datasets from the zip file, i have extracted them to databricks environment.

In [0]:
# Copy loaded ZIP files from FileStore to temporary directory
dbutils.fs.cp("dbfs:/FileStore/tables/PGYR2023_P01302025_01212025.zip", "file:/tmp/general.zip")
dbutils.fs.cp("dbfs:/FileStore/tables/PHPRFL_P01302025_01212025.zip", "file:/tmp/recipient.zip")


Out[3]: True

In [0]:
#Extract  ZIP files from temporary directory
import zipfile

# Extract general payments data from zip file 
with zipfile.ZipFile("/tmp/general.zip", 'r') as zip_ref:
    zip_ref.extractall("/tmp/openpayments_general")

# Extract recipient profiles data from zip file
with zipfile.ZipFile("/tmp/recipient.zip", 'r') as zip_ref:
    zip_ref.extractall("/tmp/openpayments_recipient")

In [0]:
# Now we have Confirm the CSV files  that were extracted
import os

print("General Payment Files:")
print(os.listdir("/tmp/openpayments_general"))


print("Recipient Files:")
print(os.listdir("/tmp/openpayments_recipient"))


General Payment Files:
['OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv', 'OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv', 'OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv', 'OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv', 'OP_PGYR2023_README_P01302025.txt']
Recipient Files:
['OP_CVRD_RCPNT_PRFL_SPLMTL_README_P01302025.txt', 'OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv']


In [0]:
import shutil
import os

# Make sure destination exists
os.makedirs("/dbfs/FileStore/tables/", exist_ok=True)

# Copy  file to FileStore
shutil.copy("/tmp/openpayments_general/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv", "/dbfs/FileStore/tables/general.csv")

Out[8]: '/dbfs/FileStore/tables/general.csv'

In [0]:
# Copy recipient file
shutil.copy("/tmp/openpayments_recipient/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv", "/dbfs/FileStore/tables/recipient.csv")


Out[9]: '/dbfs/FileStore/tables/recipient.csv'

In [0]:
# DBFS files
import os

print("  Files in /dbfs/FileStore/tables/:")
print(os.listdir("/dbfs/FileStore/tables/"))

  Files in /dbfs/FileStore/tables/:
['general.csv', 'recipient.csv']


In [0]:
# Load files using file
gen_df = spark.read.csv("file:/dbfs/FileStore/tables/general.csv", header=True, inferSchema=True)
recipient_df = spark.read.csv("file:/dbfs/FileStore/tables/recipient.csv", header=True, inferSchema=True)

# Show previews
print(" General Payments Preview:")
gen_df.show(5)

print(" Recipient Profiles Preview:")
recipient_df.show(5)

 General Payments Preview:
+-----------+----------------------+---------------------+--------------------+----------------------+----------------------------+---------------------+----------------------------+-----------------------------+---------------------------+-----------------------------+-----------------------------------------------+-----------------------------------------------+---------------+---------------+------------------+-----------------+------------------+---------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-------------------------------------+-------------------------------------+-------------------------------------+-----

Use PySpark to perform the analyses specified below on the 2023 files.

1. What is the Nature of Payments with reimbursement amounts greater than $1,000 ordered by count?

In [0]:
#What is the Nature of Payments with reimbursement amounts greater than $1,000 ordered by count?
from pyspark.sql.functions import col, count

# Filter payments > $1000, group by payment nature, count And  sort descending
gen_df.filter(col("Total_Amount_of_Payment_USDollars") > 1000) \
    .groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(count("*").alias("Count")) \
    .orderBy(col("Count").desc()) \
    .show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |Count |
+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|164092|
|Consulting Fee                                                                                                                                    |105228|
|Travel and Lodging                                                                                                                                |24738 |
|Honoraria                                                      

Payments with reimbursement amounts larger than $1,000 are dominated by compensation for services other than consulting, which includes teaching and speaker roles outside of continuing education programs (164,092 cases). Consulting fees are closely followed with 105,228 occurrences, with travel and lodging accounting for 24,738. Other important payment forms are honoraria (13,750), education (13,376), and royals or licenses (11,534). Payments for medical education programs (8,658), grants (4,922), and space rental or facility fees at teaching hospitals (4,917) are all frequent. Smaller numbers are seen for long-term medical supply loans (2,930), debt forgiveness (1,787), and food and beverage (966). Other payment kinds that have the lowest counts are gifts, acquisitions, charitable contributions, and entertainment.

2. What are the top ten Nature of Payments by count?

In [0]:
#top ten Nature of Payments by count
gen_df.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(count("*").alias("Count")) \
    .orderBy(col("Count").desc()) \
    .show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |Count   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Food and Beverage                                                                                                                                 |13378464|
|Travel and Lodging                                                                                                                                |545086  |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|236628  |
|Consulting Fee                                     

The top 10 payment types by count are Food and Beverage (13,378,464), Travel and Lodging (545,086), and Compensation for services other than consulting (236,628). Other major payments are Consulting Fees (170,630), Education (161,078), and Gifts (31,786). In addition, payments for honoraria (20,232), royalties or licenses (15,865), compensation for working as faculty (12,234), and entertainment (7,967) are among the top ten.

3. What are the top ten Nature of Payments by total amount?

In [0]:
#top ten Nature of Payments by total amount
from pyspark.sql.functions import sum as spark_sum

gen_df.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(spark_sum("Total_Amount_of_Payment_USDollars").alias("Total_Payment")) \
    .orderBy(col("Total_Payment").desc()) \
    .show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |Total_Payment       |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Royalty or License                                                                                                                                |1.19217456302E9     |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|5.946326876500002E8 |
|Consulting Fee                                                                                                                                    |5.

The top 10 payment types by total amount are Royalty or Licenseat $1.19 billion, followed by Compensation for services other than consulting at $594.63 million and Consulting Fees at $514.86 million. Other major payments include Food and Beverage ($374.49 million), Travel and Lodging ($179.55 million), and Grants ($111.89 million). Additional payments consist of Acquisitions ($71.93 million), Education ($64.70 million), Honoraria ($55.85 million), and Long-term medical supply or device loans($30.10 million).

4. What are the top ten physician specialties by total amount?

In [0]:
#top ten physician specialties by total amount
from pyspark.sql.functions import sum as spark_sum, col

gen_df.groupBy("Covered_Recipient_Specialty_1") \
    .agg(spark_sum("Total_Amount_of_Payment_USDollars").alias("Total_Payment")) \
    .orderBy(col("Total_Payment").desc()) \
    .show(10, truncate=False)

+------------------------------------------------------------------------------------------------+--------------------+
|Covered_Recipient_Specialty_1                                                                   |Total_Payment       |
+------------------------------------------------------------------------------------------------+--------------------+
|null                                                                                            |7.936674692300001E8 |
|Allopathic & Osteopathic Physicians|Orthopaedic Surgery                                         |4.0345021308999825E8|
|Allopathic & Osteopathic Physicians|Internal Medicine                                           |1.3136300307000063E8|
|Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology                            |8.979213626000014E7 |
|Allopathic & Osteopathic Physicians|Neurological Surgery                                        |8.608847857000013E7 |
|Allopathic & Osteopathic Physicians|Der

The top 10 physician specialties by total amount are topped by Orthopaedic Surgery at $403.45 million, followed by Internal Medicineat $131.36 million. Other significant specializations include Neurology under Psychiatry & Neurology ($89.79 million), Neurological Surgery ($86.09 million), and Dermatology ($83.20 million). Furthermore, Cardiovascular Disease (Internal Medicine) is responsible for $70.22 million, Hematology & Oncology (Internal Medicine) for $69.47 million, and Adult Reconstructive Orthopaedic Surgeryfor $66.77 million. Psychiatry (Psychiatry & Neurology) pays $63.30 million, whereas Null Specialty pays $79.37 million.

5. Who are the top ten physicians by total amount?

In [0]:
# top ten physicians by total amount
from pyspark.sql.functions import concat_ws, col, sum as spark_sum

# Combine first and last name into 1 column
gen_df_with_names = gen_df.withColumn(
    "Physician_Name",
    concat_ws(" ", col("Covered_Recipient_First_Name"), col("Covered_Recipient_Last_Name"))
)

# Group by Physician_Name and get total amount
gen_df_with_names.groupBy("Physician_Name") \
    .agg(spark_sum("Total_Amount_of_Payment_USDollars").alias("Total_Payment")) \
    .orderBy(col("Total_Payment").desc()) \
    .show(10, truncate=False)

+----------------+-------------------+
|Physician_Name  |Total_Payment      |
+----------------+-------------------+
|                |7.933900857900002E8|
|STEPHEN BURKHART|3.392202493E7      |
|WILLIAM BINDER  |2.943437497E7      |
|KEVIN FOLEY     |1.73059378E7       |
|IVAN OSORIO     |1.606551551E7      |
|GEORGE MAXWELL  |1.160032024E7      |
|ROBERT BOOTH    |8459167.19         |
|NEAL ELATTRACHE |7810628.2          |
|AARON ROSENBERG |6883627.290000001  |
|ROGER JACKSON   |6615711.26         |
+----------------+-------------------+
only showing top 10 rows



Stephen Burkhart leads the top ten physicians in terms of total earnings, with $33.92 million, followed by William Binder with $29.43 million and Kevin Foley with $17.31 million. Other famous physicians are Ivan Osorio($16.07 million), George Maxwell ($11.60 million), and Robert Booth ($8.46 million). Additional top physicians are Neal Elattrache ($7.81 million), Aaron Rosenberg ($6.88 million), and Roger Jackson ($6.62 million). A **Null Physician category has a total value of $793.39 million.